In [9]:
# 1. Import the necessary Packages
import sqlite3
import os
from graphviz import Digraph
from collections import defaultdict

###  NOTE: This code included in the repo for reference only as the full database has not been publicly released to maintain privacy.

In [ ]:
# Create a sfdp tree from the sql database data

# Configure root players
founders = [1, 2, 3]

# Connect to the database
conn = sqlite3.connect('/Users/firefly/Python/Sauceball Family Tree/sauceball.db')
cursor = conn.cursor()

# Get player data (omitting those who opt out)
cursor.execute("""
  SELECT PlayerID, FirstName, Nickname, Surname, Pronouns, RookieYear, RecruitedByID
  FROM Player
  WHERE RecruitedByID IS NOT NULL OR PlayerID IN (?, ?, ?)
    AND (OptOut IS NULL OR OptOut != 'Y')
""", founders)
rows = cursor.fetchall()

# Get PlayerIDs who are on a 2025 team roster
cursor.execute("""
    SELECT DISTINCT R.PlayerID
    FROM Roster R
    JOIN Team T ON R.TeamID = T.TeamID
    WHERE T.Year = 2025
""")
season_2025_players = set(row[0] for row in cursor.fetchall())

conn.close()

# Build player dict 
players = {}
for pid, fn, nn, sn, pronouns, ry, recruited in rows:
    full_name_length = len(f'{fn} {sn}')
    
    # Decide the “year string”:
    if ry == 1990:
        year_label = "1990 OG"
    else:
        year_label = f'{ry or "N/A"} Rookie'
    
    if nn:
        label = f'#{pid} {fn}\n"{nn}"\n{sn}\n{year_label}'
    else:
        if full_name_length > 15:
            label = f'#{pid} {fn}\n{sn}\n{year_label}'
        else:
            label = f'#{pid}\n{fn} {sn}\n{year_label}'
    
    players[pid] = {
        'label': label,
        'pronouns': pronouns,
        'rookie_year': ry,
        'recruited_by': recruited
    }

# Build recruitment relationships
tree = defaultdict(list)
roots = []
for pid, data in players.items():
    recruiter = data['recruited_by']
    if recruiter and recruiter in players:
        tree[recruiter].append(pid)
    else:
        roots.append(pid)
        
# Initialize Graphviz using the sfdp engine
dot = Digraph('FamilyTree', engine='sfdp', graph_attr={
    'overlap': 'false',
    'splines': 'true',
    'smoothing': 'true',
    'dpi': '150',
    'nodesep': '0.5',
    'ranksep': '0.5',
})

# Set default node attributes
dot.attr('node', fontsize='11', fontname='Silk Flower', style='filled')

for pid, d in players.items():
    recruit_count = len(tree[pid])

    # Set fill color priority:
    if pid in season_2025_players:
        fillcolor = '#AF4A3B' 
    elif pid in founders:
        fillcolor = '#E1ECBC'
    else:
        fillcolor = {
            'He/Him': '#6FA059',
            'She/Her': '#A5CC46',
            'They/Them': '#C7DD71'
        }.get(d['pronouns'], '#AAAAAA')  # Default gray
    
    dark_backgrounds = ['#24714E', '#263D41']
    fontcolor = 'white' if fillcolor in dark_backgrounds else 'black'

    # Base node style
    node_kwargs = {
        'label': d['label'],
        'fillcolor': fillcolor,
        'fontcolor': fontcolor,
        'style': 'filled',
        'shape': 'ellipse',
        'fontsize': '11',
        'penwidth': '1',
        'fixedsize': 'false',
        'margin': '0'
    }

    # 🌟 Founders
    if pid in founders:
        node_kwargs.update({
            'fontsize': '16',
            'penwidth': '3',
            'width': '2.25',
            'height': '2.25',
            'shape': 'circle',
            'fixedsize': 'true',
            'margin': '0'
        })

    # 🎯 Super recruiters
    elif recruit_count >= 4:
        node_kwargs.update({
            'fontsize': '13',
            'width': '2',
            'height': '2',
            'shape': 'circle',
            'fixedsize': 'true',
            'margin': '0'
        })

    dot.node(str(pid), **node_kwargs)

# Add recruitment edges
dot.attr('edge', color='#6E695C', penwidth='4', arrowhead='dot')
for recruiter, recruits in tree.items():
    for recruit in recruits:
        dot.edge(str(recruiter), str(recruit))

# Render Graphics
folder = 'image'
filename = 'familytree'

png_path = os.path.join(folder, filename)
dot.format = 'png'
dot.render(png_path, view=True)

svg_path = os.path.join(folder, filename)
dot.format = 'svg'
dot.render(svg_path, view=False)